Overview


Let $T>0$ be the terminal time, and ${W}_{t_0 \leq t \leq T}$ be a standard $d$-dimensional Brownian motion. Consider the following linear BSDE

$-dY_t  = (A_t Y_t + Z_t.B_t + C_t)dt - Z_t.dW_t, Y_T = \xi. $ 

where $Y_t$ and $Z_t$ are of dimension $d \times 1$, $A_t$ is $d \times d$, $B_t$ is $d \times 1$, and $C_t$ is $d \times 1$. The notation $Z_t.B_t$ (with a dot) means element-wise multiplication.

From Pham (2009), we know that the solution $Y_t$ is given by: 

$\Gamma_t Y_t = E[\Gamma_T \xi + \int_t^T \Gamma_s C_s ds|F_t]$, where $d\Gamma_t = \Gamma_t (A_t dt + B_t.dW_t), \Gamma_0 = 1$. 

We try to find this solution both analytically and via simulation.

First we will set up the problem and use gradient descent to determine the optimal $Y_0$ and $Z_0$ in the case where $A,B,C$ are all constant. 


In [5]:
### Packages
#import Pkg; Pkg.build("DifferentialEquations") 
import Pkg; Pkg.add("DifferentialEquations")
import Pkg; Pkg.add("IterTools")
import Pkg; Pkg.add("DiffEqBase")

using Distributed
using DifferentialEquations
using Flux
using DifferentialEquations.EnsembleAnalysis
using IterTools: ncycle
using Flux: @epochs
using DiffEqSensitivity
using Statistics
using DiffEqFlux
using DiffEqBase.EnsembleAnalysis


   Building SLEEFPirates → `~/.julia/packages/SLEEFPirates/jGsib/deps/build.log`
   Building Random123 ───→ `~/.julia/packages/Random123/Y2Du3/deps/build.log`
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]


In [6]:

### Setting up the problem
T = 1.0f0
tspan = (0.0f0,T)
d = 5 #number of dimensions
m=[1.0,1.0,1.0,1.0,1.0] # mean for the ith dimension
v=[1.0,1.0,1.0,1.0,1.0] # variance for the ith dimension for Y_T
A = [1.0 0.0 0.0 0.0 0.0;
    0.0 1.0 0.0 0.0 0.0;
    0.0 0.0 1.0 0.0 0.0;
    0.0 0.0 0.0 1.0 0.0;
    0.0 0.0 0.0 0.0 1.0] # dxd matrix
B = [1.0,1.0,1.0,1.0,1.0] # dx1 matrix
C = [1.0,1.0,1.0,1.0,1.0] # dx1 matrix
u0 = [10.0,10.0,10.0,10.0,10.0] # initial guess for Y_0
Z = [-5.0,-5.0,-5.0,-5.0,-5.0] # Guess for Z
p = [Z;u0] # parameters to update (specifically initial position of Y and Z)

10-element Array{Float64,1}:
 -5.0
 -5.0
 -5.0
 -5.0
 -5.0
 10.0
 10.0
 10.0
 10.0
 10.0

In [7]:
## Setting up the drift and stochastic portions

function drift_nn!(du,u,p,t)
    du[1:d] = -(A*u + B.*p[1:d] + C)
end

function stoch_nn!(du,u,p,t)
    du[1:d] = p[1:d]
end

probSDE = SDEProblem(drift_nn!,stoch_nn!,p[end-d+1:end],tspan,p=p)
tmp_prob = remake(probSDE,p=p,u0=p[end-d+1:end])
ensemble_prob = gpu(EnsembleProblem(tmp_prob))
@time sol = solve(ensemble_prob,EM(),saveat = T,EnsembleThreads(), trajectories=1000,dt=0.05) # now uses EM() instead of EM()
arrsol = Array(sol);

  2.235498 seconds (5.16 M allocations: 233.813 MiB, 1.96% gc time)


<font color=blue>`EulerHeun` is for Stratonovich stochastic integrals. The analogous method for the Ito integrals that we consider here is `EM` (See [here](https://diffeq.sciml.ai/v3.0/solvers/sde_solve.html)). Obviously, there was no way to know which definition of stochastic integral we were using. I've changed all `EulerHeun` to `EM`.</font>


In [9]:
function loss(p)
    tmp_prob = remake(probSDE,p=p,u0=p[end-d+1:end])
    #ensemble_prob = gpu(EnsembleProblem(tmp_prob))
    #sim = solve(ensemble_prob,EM(),saveat = T,EnsembleThreads(),trajectories=1000, dt=0.05) 
    ensemble_prob = EnsembleProblem(tmp_prob)
    sim = solve(ensemble_prob,EM(),saveat = T,EnsembleThreads(),trajectories=1000, dt=0.05) # See comment in cell below
    arrsol = Array(sim)
    loss = sum((m .- mean(arrsol,dims=3)[:,2]).^2) + sum((v .- var(arrsol,dims=3)[:,2]).^2)
end

display(p)
display(loss(p))

10-element Array{Float64,1}:
 -5.0
 -5.0
 -5.0
 -5.0
 -5.0
 10.0
 10.0
 10.0
 10.0
 10.0

657.0904741240597

<font color=blue> Does `EnsembleThreads()` work faster if `ensemble_prob` is in the gpu? I would have imagined that it would move the data back and forth between the gpu and the processor threads. I will need a refresher from you on how the parallelization/gpu works. For `EnsembleThreads()`, did I have configure anything in Julia before running the code (like that initialization script we had discussed)?</font>

In [ ]:
### Training the model

opt = ADAM(1.0)
@time res = DiffEqFlux.sciml_train(loss,p,opt, maxiters = 20)

opt = ADAM(0.1)
@time res2 = DiffEqFlux.sciml_train(loss,res.minimizer,opt, maxiters = 20)

opt = ADAM(0.01)
@time res3 = DiffEqFlux.sciml_train(loss,res2.minimizer,opt, maxiters = 50)

display(res3.minimizer)

As you can see, the solutions for each path are similar, with $Z = 1.35$, $Y_0 = 8.80$. This is because each path has identical values for $A$, $B$, $C$ and the distribution of $Y_T$. To present the mean path:   

In [14]:
probSDE = SDEProblem(drift_nn!,stoch_nn!,res3.minimizer[end-d+1:end],tspan,p=res3.minimizer)
tmp_prob = remake(probSDE,p=res3.minimizer,u0=res3.minimizer[end-d+1:end])
ensemble_prob = gpu(EnsembleProblem(tmp_prob))
@time sol = solve(ensemble_prob,EM(),EnsembleThreads(), trajectories=1000,dt=0.05)
arrsol = Array(sol)
display(mean(arrsol,dims=3)) # this gives us the mean value of Y_t at each point in time with the final point being Y_T. 

5×21×1 Array{Float64,3}:
[:, :, 1] =
 6.64572  6.20082  5.78937  5.37712  …  1.39464  1.21783  1.05568  0.859898
 6.58445  6.14199  5.71626  5.31466     1.4313   1.23961  1.06485  0.889151
 6.71965  6.26219  5.83435  5.43347     1.44859  1.24677  1.0718   0.905797
 6.65203  6.20328  5.76715  5.37565     1.3955   1.19475  1.00728  0.836625
 6.47671  6.02826  5.62071  5.21583     1.3914   1.20177  1.04469  0.881059

  0.046737 seconds (617.01 k allocations: 44.686 MiB)


Now we will revisit the problem to try to find an analytic solution to ensure that our values for $Y_t$ and $Z_t$ are accurate besides just looking at the loss function. 

The unique solution $Y$ to the linear BSDE is given by $\Gamma_t Y_t = E[\Gamma_T \xi + \int_t^T \Gamma_s C_s ds|F_t],$ where $\Gamma$ is the adjoint process solution to the linear SDE $d\Gamma_t =\Gamma_t(A_tdt+B_t.dW_t), \Gamma_0 = 1.$  

We use the short-hand notation $E_t[\cdot]= E[\cdot|F_t]$. We can simplify saying: $\Gamma_t Y_t = E_t[\Gamma_T] E_t[\xi] + \int_t^T E_t[\Gamma_s C_s] ds$. 

We can rewrite $\Gamma_t$ in two ways: either

(1) $\Gamma_t = exp (\int_0^t B_s dW_s - \frac{1}{2} \int_0^t {B_s}^2 ds + \int_0^t A_s ds)$.

(2) $\Gamma_t = exp (B_t dW_s + (A_t - \frac{1}{2} B_t^2)dt)$. <font color=blue> I don't think (2) makes sense -- the exponential of dW and dt is not well-defined</font>

Using (2), when $B_t,A_t,C_t$ are constant, we can say: 

$\Gamma_t = E_t[exp(B dW_s + (A - \frac{1}{2} B^2)dt)]$.

$\Gamma_t = exp((B + A - \frac{1}{2} B^2)t)$

Illustrating this analytically: 

In [17]:
T = 1.0f0
tspan = (0.0f0,T)

gamma_0 = [1.0, 1.0, 1.0, 1.0, 1.0]

function drift_gamma!(du,u,p,t)
    #du[1:d] = [A[1,1] A[2,2] A[3,3] A[4,4] A[5,5]] 
    du[1:d] = A*u
end

function stoch_gamma!(du,u,p,t)
    #du[1:d] = B
     du[1:d] = B.*u
end

prob = SDEProblem(drift_gamma!,stoch_gamma!,gamma_0,tspan)
ensemble_prob = gpu(EnsembleProblem(prob))
@time sol = solve(ensemble_prob,EM(),EnsembleThreads(), trajectories=10000,dt=0.005)
arrsol = Array(sol)
display(mean(arrsol,dims=3))

  5.379063

5×201×1 Array{Float64,3}:
[:, :, 1] =
 1.0  1.00687  1.01447  1.02165  …  4.44677  4.48434  4.51715  4.54567
 1.0  1.00682  1.01459  1.02226     4.50613  4.54165  4.57592  4.60716
 1.0  1.00656  1.01445  1.02179     4.29179  4.32905  4.3645   4.40851
 1.0  1.00703  1.01535  1.02475     4.4125   4.45826  4.50048  4.53361
 1.0  1.00653  1.01404  1.02227     4.33236  4.36162  4.3954   4.41748

 seconds (36.22 M allocations: 1.986 GiB, 15.51% gc time)


In [18]:
ℯ^(([A[1,1] A[2,2] A[3,3] A[4,4] A[5,5]]' .- (B.^2 ./ 2) .+ B)*T)[1]

4.4816890703380645

For this example (and all others where A and B aren't a function of t), the math works out. 

Now, back to our original reason for understanding $\Gamma_t$. 

We say: $\Gamma_t Y_t = E[\Gamma_T \xi + \int_t ^T \Gamma_s C_s ds|F_t]$ 

Substituting our value for $\Gamma_t$ into the equation (and assuming $A, B, C$ are constant) we get: 

$exp((B + A - \frac{1}{2} B^2)t) Y_t = exp((B + A - \frac{1}{2} B^2)T) E_t[\xi] + \int_t^T C exp((B + A - \frac{1}{2} B^2)s)  ds$

When t = T we get: 

$Y_T = \xi$, which makes sense.

When t = 0 we get: 

$Y_0 = exp(B+A-\frac{1}{2}B^2) T)  E_t[\xi] + C \int_0^T  exp{(B + A - \frac{1}{2} B^2)s}  ds$


Solving for the case A = B = C = T = 1 and $\xi \sim N(1,1)$:

$Y_0 =  exp(3/2) + \int_0^1 exp(3s/2) ds$. 

$Y_0 = exp(3/2) + (2exp(3/2) - 2)/3$, which is the solution we get from simulation. 



In [94]:
ℯ^(3/2) + (2*ℯ^(3/2)-2)/3

6.802815117230107

In [97]:
res3.minimizer

10-element Array{Float64,1}:
 1.4508912511387506
 1.5239255366211126
 1.2520843697505666
 1.3821145007148052
 1.2465729407748982
 6.726154133924283
 6.987599800913823
 6.547568369180367
 6.6466004804729275
 6.536089299876388

Now let's consider the case where $A$, $B$, or $C$ are not constant with respect to time. We'll work case by case:

Case 1: $C$ not constant with respect to time. 

First, let's consider the case where $A$ and $B$ are constant with respect to time. The solution for $\Gamma_t$ is identical because $\Gamma_t$ is not a function of $C$. 

We say: $\Gamma_t Y_t = E[\Gamma_T \xi + \int_t ^T \Gamma_s C_s ds|F_t]$ 

Substituting our value for $\Gamma_t$ into the equation (and assuming $A, B, C$ are constant) we get: 

$exp((B + A - \frac{1}{2} B^2)*t) Y_t = exp((B + A - \frac{1}{2} B^2)*T) * \xi + * \int_t^T C* exp((B + A - \frac{1}{2} B^2)*s)  ds$

Let $A = B= 1$. Let $\Xi ~ N(1,1)$. Let $C_t = t$ for example: 

$exp(3t/2) Y_t = exp(3T/2) * \Xi +  \int_t^T s * exp(3s/2)  ds$

$exp(3t/2) Y_t = exp(3/2) *\Xi + ((6T - 4)exp(3T/2) + (4-6t)exp(3t/2))/9$

For $t = T$, the solution is $Y_t = \Xi$.

For $t = 0$, the solution is $Y_0 = exp(3/2) + (2 exp(3/2) - 2(exp(0))/9$




In [99]:
## Setting up the drift and stochastic portions

function drift_nn!(du,u,p,t)
    du[1:d] = -(A*u + B.*p[1:d] .+ t)
end

function stoch_nn!(du,u,p,t)
    du[1:d] = p[1:d]
end

probSDE = SDEProblem(drift_nn!,stoch_nn!,p[end-d+1:end],tspan,p=p)
tmp_prob = remake(probSDE,p=p,u0=p[end-d+1:end])
ensemble_prob = gpu(EnsembleProblem(tmp_prob))
@time sol = solve(ensemble_prob,EM(),saveat = T,EnsembleThreads(), trajectories=1000,dt=0.05)
arrsol = Array(sol)

function loss(p)
    tmp_prob = remake(probSDE,p=p,u0=p[end-d+1:end])
    ensemble_prob = gpu(EnsembleProblem(tmp_prob))
    sim = solve(ensemble_prob,EM(),saveat = T,EnsembleThreads(),trajectories=1000, dt=0.05)
    arrsol = Array(sim)
    loss = sum((m .- mean(arrsol,dims=3)[:,2]).^2) + sum((v .- var(arrsol,dims=3)[:,2]).^2)
end

display(p)
display(loss(p))

10-element Array{Float64,1}:
 10.0
 10.0
 10.0
 10.0
 10.0
 -5.0
 -5.0
 -5.0
 -5.0
 -5.0

10100.933489964815

  3.429308 seconds (6.26 M allocations: 292.293 MiB, 1.40% gc time)


In [100]:
### Training the model

opt = ADAM(1.0)
@time res = DiffEqFlux.sciml_train(loss,p,opt, maxiters = 20)

opt = ADAM(0.1)
@time res2 = DiffEqFlux.sciml_train(loss,res.minimizer,opt, maxiters = 20)

opt = ADAM(0.01)
@time res3 = DiffEqFlux.sciml_train(loss,res2.minimizer,opt, maxiters = 50)

display(res3.minimizer)

loss: 27.6: 100%|███████████████████████████████████████| Time: 0:01:24


100.164111 seconds (291.03 M allocations: 19.809 GiB, 6.42% gc time)


loss: 3.78: 100%|███████████████████████████████████████| Time: 0:01:25


 89.666151 seconds (273.06 M allocations: 18.979 GiB, 7.47% gc time)


loss: 0.197: 100%|██████████████████████████████████████| Time: 0:03:39


10-element Array{Float64,1}:
 1.3124705366280471
 1.3277106112429153
 1.4195204402488046
 1.4488634906156754
 1.474578704509947
 6.145850029386997
 6.188714543551074
 6.2487628910875115
 6.291254170056632
 6.266248937122043

223.874741 seconds (682.64 M allocations: 47.448 GiB, 7.02% gc time)


In [104]:
ℯ^(3/2) + (2*ℯ^(3/2)-2)/9

5.255397752635412

The two values don't match up entirely, however, that is unclear if it is because $Z_t$ is supposed to be not a constant in this scenario, but I have constrained it to be a constant, or if I haven't let the parameters update enough, or something else. 